In [ ]:
%pip install google-genai
%pip install Pillow
%pip install matplotlib
%pip install dotenv

In [ ]:
from google import genai
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file")    

# Configure the client with your API key
client = genai.Client(api_key=api_key)

In [ ]:
prompt = """Create a photorealistic image of an orange cat
with a green eyes, sitting on a couch."""

In [ ]:
MODEL="gemini-2.5-flash-image"

In [ ]:
# Call the API to generate content
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
)

# The response can contain both text and image data.
# Iterate through the parts to find and save the image.
for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        image = Image.open(BytesIO(part.inline_data.data))
        # image.save("cat.png")
        # Display the image in a cell
        plt.imshow(image)
        plt.axis("off") # hide the axes
        plt.show()


In [ ]:
# Image-only-outputs

# Call the API to generate content
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config={"response_modalities": ["Image"]},
)

# for part in response.candidates[0].content.parts:
part = response.candidates[0].content.parts[0]
if part is not None and part.inline_data is not None:
    image = Image.open(BytesIO(part.inline_data.data))
    plt.imshow(image)
    plt.axis("off") # hide the axes
    plt.show()

In [ ]:
# Aspect ratios
aspect_ratios = [
    "1:1", 
    "2:3", 
    "3:2", 
    "3:4", 
    "4:3", 
    "4:5", 
    "5:4", 
    "9:16", 
    "16:9", 
    "21:9"
]

In [ ]:
for ratio in aspect_ratios:
    response = client.models.generate_content(
        model=MODEL,
        contents=prompt,
        config={
            "response_modalities": ["Image"],
            "image_config": {"aspect_ratio": ratio}
        },
    ) 

    part = response.candidates[0].content.parts[0]
    if part is not None and part.inline_data is not None:
        print("Aspect ratio: ", ratio)
        image = Image.open(BytesIO(part.inline_data.data))
        plt.imshow(image)
        plt.axis("off") # hide the axes
        plt.show()  

In [ ]:
# Converational Image editing

# original cat image
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config={
        "response_modalities": ["Image"],
    },
)

part = response.candidates[0].content.parts[0]
image = Image.open(BytesIO(part.inline_data.data)) if part is not None and part.inline_data is not None else None 
plt.imshow(image)
plt.axis("off") # hide the axes
plt.show()  

In [ ]:
# first image edit
image2: None = None
if image:
    # Create a chat
    chat = client.chats.create(
        model=MODEL
    )

    # Make the first image edit
    response1 = chat.send_message(
        [
            "Change the cat to a bengal cat, leave everything else the same",
            image,
        ]
    )

    part2 = response1.candidates[0].content.parts[0]
    if part2 is not None and part2.inline_data is not None:
        image2 = Image.open(BytesIO(part2.inline_data.data))
        plt.imshow(image2)
        plt.axis("off") # hide the axes
        plt.show()  

In [ ]:
image3 = None
if image2:
    # Make the first image edit
    response2 = chat.send_message(
        [
            "The cat should wear a funny party hat",
            image2,
        ]
    )

    part3 = response2.candidates[0].content.parts[0]
    if part3 is not None and part3.inline_data is not None:
        image3 = Image.open(BytesIO(part3.inline_data.data))
        plt.imshow(image3)
        plt.axis("off") # hide the axes
        plt.show() 

In [ ]:
image4 = None
if image3:
    # Make the first image edit
    response3 = chat.send_message(
        [
            "The cat should wear a shade",
            image3,
        ]
    )

    part4 = response3.candidates[0].content.parts[0]
    if part4 is not None and part4.inline_data is not None:
        image4 = Image.open(BytesIO(part4.inline_data.data))
        plt.imshow(image4)
        plt.axis("off") # hide the axes
        plt.show() 